In [11]:
from context import *
from torchclifford import circuit as tcircuit
from torchclifford import paulialg as tpaulialg
from torchclifford import stabilizer as tstabilizer

`circuit.py` contains **three classes**: `CliffordGate`, `CliffordLayer` and `CliffordCircuit`

# CliffordGate

## Basic Usage

`CliffordGate` class represents the Clifford gates. 
- `Note:` all TorchClifford operations are done along side PyClifford operations to illustrate their identical signatures. The only difference is the device kwarg.
- **Attribute**:
    - qubits: a tuple containing the number of qubits that it acts on 
    - n: number of qubits it acts on
    - generator(Pauli): the generator for $C_{\pi/4}$ rotation. `Default: None`
    - forward_map(CliffordMap): forward Clifford Map. `Default: None`
    - backward_map(CliffordMap): backward Clifford Map. `Default: None`
- **Methods**:
    - `CliffordGate.set_generator(Pauli)`
    - `CliffordGate.set_forward_map(CliffordMap)`
    - `CliffordGate.set_backward_map(CliffordMap)` **TODO: we need to add a handle to prevent user set conflict Clifford maps**
    - `CliffordGate.copy()`: return a copy of the Clifford gate
    - `CliffordGate.independent_from(Other_gate)`: if there is no overlap between acting qubits, then they are independent.
    - `CliffordGate.compile()`: build (forward&backward) Clifford map representation. If generator is given, then it will be converted to forward/backward map; if forward map is given, the backward map will be calculated; if backward map is given, the forward map will be calculated
    - `CliffordGate.forward(obj)`: forward transformation of the object. `obj` can be Pauli, PauliList, PauliPolynomial, StabilizerState.

CliffordGate is the low-level API(class) for Clifford gates. One can create a **null** Clifford gate by sepecifying what are the qubits this gate will act on:

Example: `circuit.CliffordGate(1,2)` will create a null gate acting on qubit 1, and 2. Null gate does not contain any `forward_map` or `backward_map`

In [6]:
gate = circuit.CliffordGate(1,2)
tgate = tcircuit.CliffordGate(1,2)

In [8]:
print("gate acts on: {}, and there are {} qubits the gates will act on.".format(
gate.qubits, gate.n
))
print("torch gate acts on: {}, and there are {} qubits the gates will act on.".format(
tgate.qubits, tgate.n
))

gate acts on: (1, 2), and there are 2 qubits the gates will act on.
torch gate acts on: (1, 2), and there are 2 qubits the gates will act on.


There are three ways to equip the null gate with Clifford map:
1. One can use `gate.set_generator(Pauli)`. This will create a $\pi/2$ rotation generated by the pauli string.
2. One can use `gate.set_forward_map(CliffordMap)`
3. One can use `gate.set_backward_map(CliffordMap)`

In [10]:
gate.set_generator(paulialg.pauli("XX"))
tgate.set_generator(tpaulialg.pauli("XX"))

In [21]:
gate.set_forward_map(stabilizer.random_clifford_map(2))
tgate.set_forward_map(tstabilizer.random_clifford_map(2))

In [22]:
gate.set_backward_map(stabilizer.random_clifford_map(2))
tgate.set_backward_map(tstabilizer.random_clifford_map(2))

`gate.compile()` will construct both `forward_clifford_map` and `backward_clifford_map`. Compilation is not necessary!

`gate.copy()` will return a copy of the `CliffordGate`

One can apply Clifford Gate $U$ or $U^{\dagger}$ to: Pauli strings, Stabilizer States by `gate.forward(obj)` and `gate.backward(obj)`

In [23]:
psi = stabilizer.random_clifford_state(5)
tpsi = tstabilizer.random_clifford_state(5)

In [24]:
print(psi)
print()
print(tpsi)

StabilizerState(
   +YXXYX
   +YZZZZ
   +XYXIX
   +IZYYI
   +ZZYIZ)

StabilizerState(
   +XIIII
   +IYIII
   +IIYII
   +IIIYI
   +IIIIY)


In [25]:
gate = circuit.CliffordGate(1,2)
tgate = tcircuit.CliffordGate(1,2)
gate.set_generator(paulialg.pauli("XX"))
tgate.set_generator(tpaulialg.pauli("XX"))

In [26]:
print(gate.forward(psi))
print()
print(tgate.forward(tpsi))

StabilizerState(
   +YXXYX
   +YZZZZ
   +XZIIX
   +IZYYI
   +ZZYIZ)

StabilizerState(
   +XIIII
   +IZXII
   +IXZII
   +IIIYI
   +IIIIY)


In [28]:
print(psi)
print()
print(tpsi)

StabilizerState(
   +YXXYX
   +YZZZZ
   +XZIIX
   +IZYYI
   +ZZYIZ)

StabilizerState(
   +XIIII
   +IZXII
   +IXZII
   +IIIYI
   +IIIIY)


We see the wavefunction $|\psi\rangle$ has been changed **in-place**. And the gate only changes qubit 1&2.

In [29]:
print(gate.backward(psi))
print()
print(tgate.backward(tpsi))

StabilizerState(
   +YXXYX
   +YZZZZ
   +XYXIX
   +IZYYI
   +ZZYIZ)

StabilizerState(
   +XIIII
   +IYIII
   +IIYII
   +IIIYI
   +IIIIY)


If two Clifford gate acting on different qubits, then they are independent. We can check dependency by `gate.independent_from(other_gate)`

In [30]:
gate1 = circuit.CliffordGate(0,1)
gate2 = circuit.CliffordGate(2,3)
print(gate1.independent_from(gate2))
tgate1 = tcircuit.CliffordGate(0,1)
tgate2 = tcircuit.CliffordGate(2,3)
print(tgate1.independent_from(tgate2))

True
True


In [31]:
gate1 = circuit.CliffordGate(0,1)
gate2 = circuit.CliffordGate(1,2)
print(gate1.independent_from(gate2))
tgate1 = tcircuit.CliffordGate(0,1)
tgate2 = tcircuit.CliffordGate(1,2)
print(tgate1.independent_from(tgate2))

False
False


# CliffordLayer

`CliffordLayer` has the following attributes:
- gates(list): contains a list of `CliffordGate`
- prev_layer: Default `None`. It will be automatically setup in `CliffordCircuit`
- next_layer: Default `None`. It will be automatically setup in `CliffordCircuit`
- forward_map
- backward_map

`CliffordLayer` can take `CliffordGates` by `layer.take(gate)`:

In [32]:
circlayer = circuit.CliffordLayer()
tcirclayer = tcircuit.CliffordLayer()

In [33]:
circlayer.take(circuit.CliffordGate(0,1))
tcirclayer.take(tcircuit.CliffordGate(0,1))

In [34]:
circlayer.take(circuit.CliffordGate(1,3))
tcirclayer.take(tcircuit.CliffordGate(1,3))

In [35]:
print(circlayer)
print(tcirclayer)

|[0,1][1,3]|
|[0,1][1,3]|


In [37]:
print(circlayer.gates)
print(tcirclayer.gates)

[[0,1], [1,3]]
[[0,1], [1,3]]


In [38]:
psi = stabilizer.random_clifford_state(4)
print(psi)
print()
tpsi = tstabilizer.random_clifford_state(4)
print(tpsi)

StabilizerState(
   +ZXZI
   +XXYY
   +IYXY
   +YYIY)

StabilizerState(
   +YIII
   +IYII
   +IIXI
   +IIIY)


In [39]:
print(circlayer.forward(psi))
print()
print(tcirclayer.forward(tpsi))

StabilizerState(
   -XXZZ
   +ZYYY
   -XIXY
   +ZXIX)

StabilizerState(
   -YIII
   -IYII
   +IIXI
   +IIIX)


<div class="alert alert-block alert-success">
If the forward map and backward map for a gate is Null, then it will be assigned a <b> different </b>random clifford map each time when use calls forward() or backward()
</div>

# CliffordCircuit

`CliffordCircuit` is the high level API to assemble gates and layers, and it **automatically** calculate number of qubits in the system
It has attributes:
- first_layer
- last_layer
- forward_map
- backward_map

When `CliffordCircuit` is initialized, a null `CliffordLayer()` will be initiated.
`CliffordCircuit` can take `CliffordGate` by `circ.take(gate)`.

If the gate is independent from the current layer, it will be added. Otherwise, the circuit will create a new layer and add the gate.

In [47]:
circ = circuit.CliffordCircuit(11)
tcirc = tcircuit.CliffordCircuit()

In [48]:
print(circ.take(circuit.CliffordGate(0,1)))
print(tcirc.take(tcircuit.CliffordGate(0,1)))

CliffordCircuit(
  |[0,1]|)
CliffordCircuit(
  |[0,1]|)


In [49]:
print(circ.take(circuit.CliffordGate(9,10)))
print(tcirc.take(tcircuit.CliffordGate(9,10)))

CliffordCircuit(
  |[0,1][9,10]|)
CliffordCircuit(
  |[0,1][9,10]|)


In [50]:
print(circ.N)
print(tcirc.N)

11
11


In [51]:
psi = stabilizer.ghz_state(11)
tpsi = tstabilizer.ghz_state(11)

In [52]:
print(circ.forward(psi))
print()
print(tcirc.forward(tpsi))

StabilizerState(
   +XXIIIIIIIII
   +ZZZIIIIIIII
   +IIZZIIIIIII
   +IIIZZIIIIII
   +IIIIZZIIIII
   +IIIIIZZIIII
   +IIIIIIZZIII
   +IIIIIIIZZII
   -IIIIIIIIZIZ
   -IIIIIIIIIYZ
   -ZYXXXXXXXXX)

StabilizerState(
   +XYIIIIIIIII
   -IYZIIIIIIII
   +IIZZIIIIIII
   +IIIZZIIIIII
   +IIIIZZIIIII
   +IIIIIZZIIII
   +IIIIIIZZIII
   +IIIIIIIZZII
   -IIIIIIIIZXI
   -IIIIIIIIIXX
   +ZZXXXXXXXZY)


In [57]:
print(psi.entropy([1,2,3,4,5]))

2


In [44]:
circuit.clifford_rotation_gate([0,2,3,0,7]).generator

-iYZ

In [49]:
circ = circuit.identity_circuit(5).take(circuit.clifford_rotation_gate([0,2,3,0,5])).take(circuit.clifford_rotation_gate([0,0,0,1,3,4]))

In [56]:
circ.compile(5)

CliffordCircuit(
  |[1,2][3,4]|)

In [57]:
circ.forward_map

CliffordMap(
  X0-> +XIIII
  Z0-> +ZIIII
  X1-> +IZZII
  Z1-> -IXZII
  X2-> -IYYII
  Z2-> +IIZII
  X3-> +IIIXI
  Z3-> -IIIYZ
  X4-> +IIIXY
  Z4-> +IIIIZ)

# $Z_2$ rank

In [80]:
def testz2rank(mat):
    '''
    Parameters:
    mat: int matrix - input binary matrix.
        caller must ensure that mat contains only 0 and 1.
        mat is destroyed upon output! 
    Returns:
    r: int - rank of the matrix under Z2 algebra.'''
    nr, nc = mat.shape # get num of rows and cols
    r = 0 # current row index
    for i in range(nc): # run through cols
        if r == nr: # row exhausted first
            return r # row rank is full, early return
        if mat[r, i] == 0: # need to find pivot
            found = False # set a flag
            for k in range(r + 1, nr):
                if mat[k, i]: # mat[k, i] nonzero
                    found = True # pivot found in k
                    break
            if found: # if pivot found in k
                # swap rows r, k
                for j in range(i, nc):
                    tmp = mat[k,j]
                    mat[k,j] = mat[r, j]
                    mat[r,j] = tmp
            else: # if pivot not found
                continue # done with this col
        # pivot has moved to mat[r, i], perform GE
        for j in range(r + 1, nr):
            if mat[j, i]: # mat[j, i] nonzero
                mat[j, i:] = (mat[j, i:] + mat[r, i:])%2
        r = r + 1 # rank inc
    # col exhausted, last nonvanishing row indexed by r
    return r

In [90]:
binary_mat = np.array([[0,1,0,0,1],[1,0,1,1,0],[1,1,1,1,1],[0,0,0,0,1]])

In [91]:
testz2rank(binary_mat)

3

In [92]:
binary_mat

array([[1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])